## Accenture - Supply Chain


In [73]:
# Packaging management
import numpy as np
import pandas as pd
import csv 
from sklearn.linear_model import LinearRegression as lm
import statsmodels

In [112]:
## Distance matrix preparation

## Read raw data from csv files
pd_cities_raw = pd.read_csv("datas/cities_data.csv" ,sep=";")

city_list_s = pd_cities_raw['city_from_name'].unique()
city_list_d = pd_cities_raw['city_to_name'].unique()
all_cities= list(set(city_list_s) | set(city_list_d))

city_dict = {all_cities[i]:i for i in range(0,len(all_cities))}
n_cities = len(all_cities)

distance_mat = [[0]*n_cities]*n_cities


for index, row in pd_cities_raw.iterrows():
    s = city_dict[row["city_from_name"]]
    d = city_dict[row["city_to_name"]]
    distance = row["distance"]
    distance_mat[s][d] = distance

In [113]:
## Product Matrix

pd_products_raw = pd.read_csv("datas/product_attributes.csv", sep=",")

##dataframe quering example  !(return a string value)
# pd_products_raw.loc[pd_products_raw["product_id"]==1672707]["material_handling"].to_string(index=False)


In [116]:
all_cities

['Helsinki',
 'Warsaw',
 'Marseille',
 'Malmö',
 'Valencia',
 'Copenhagen',
 'Berlin',
 'Bremen',
 'Bucharest',
 'Rotterdam',
 'Hanover',
 'Bordeaux',
 'Lisbon',
 'Hamburg',
 'Budapest',
 'Rome',
 'Paris',
 'Amsterdam',
 'Zaragoza',
 'Cologne',
 'Munich',
 'Prague',
 'Athens',
 'Liege',
 'Milan',
 'Madrid',
 'Porto',
 'Vienna',
 'Barcelona',
 'Turin',
 'Bratislava',
 'Lille',
 'Stockholm',
 'Lyon',
 'Naples',
 'Venlo',
 'Dusseldorf']

In [156]:
## Probability dictionary calculation
pd_orders_raw=pd.read_csv("datas/orders.csv",sep=";")

pd_orders_raw.loc[pd_orders_raw["origin_port"] == "BCN", "origin_port"] = "Barcelona"
pd_orders_raw.loc[pd_orders_raw["origin_port"] == "ATHENAS", "origin_port"] = "Athens"


cities_origin = pd_orders_raw['origin_port'].unique()
cities_hub = pd_orders_raw['logistic_hub'].unique()
cities_customer =  pd_orders_raw['customer'].unique()
pd_origin_count = pd_orders_raw.groupby(['origin_port', 'late_order']).size()
pd_hub_count = pd_orders_raw.groupby(['logistic_hub', 'late_order']).size()
pd_custormer_count = pd_orders_raw.groupby(['customer', 'late_order']).size()

delay_rate_origin = {all_cities[i]:0 for i in range(0,len(all_cities))}
delay_rate_hub = {all_cities[i]:0 for i in range(0,len(all_cities))}
delay_rate_customer = {all_cities[i]:0 for i in range(0,len(all_cities))}




In [179]:
d_origin_count = pd_orders_raw.groupby(['origin_port']).size()
temp = d_origin_count.to_dict()
temp
print(temp["Barcelona"])

21556


In [163]:
origin_count_dict = pd_origin_count.to_dict()
for item in cities_origin:
    count_true = origin_count_dict
    print(count_true)

<class 'str'>


TypeError: bad operand type for unary +: 'str'

In [168]:
for key, values in origin_count_dict:
    print(key)
    print(values)

Athens
False
Athens
True
Barcelona
False
Barcelona
True
Rotterdam
False
Rotterdam
True


In [149]:
test= pd_origin_count.to_dict()

In [151]:
test[('Barcelona',True)]

5825

In [143]:
pd_origin_count
for city in pd_origin_count["origin_port"]:
    print(pd_origin_count.loc[pd_origin_count["origin_port"]==city])

KeyError: 'origin_port'

In [142]:
pd_origin_count

origin_port  late_order
Athens       False         17229
             True          10300
Barcelona    False         15731
             True           5825
Rotterdam    False         54160
             True          11031
dtype: int64

In [130]:
pd_hub_count

logistic_hub  late_order
Bratislava    False          3005
              True           1486
Dusseldorf    False         10217
              True           2301
Hamburg       False         17140
              True           4340
Liege         False          3612
              True           1029
Lille         False          6563
              True           3685
Rome          False          9251
              True           3209
Venlo         False         19259
              True           4441
Warsaw        False         12207
              True           3596
Zaragoza      False          5128
              True           2758
dtype: int64

In [131]:
pd_custormer_count

customer    late_order
Amsterdam   False         4354
            True          1059
Athens      False         2309
            True           834
Barcelona   False         3395
            True           991
Berlin      False         7896
            True          1923
Bordeaux    False         1103
            True           518
Bremen      False         1328
            True           284
Bucharest   False         2589
            True           763
Budapest    False         2152
            True           568
Cologne     False         3189
            True           715
Copenhagen  False         3576
            True          1301
Hanover     False         1828
            True           422
Helsinki    False         1077
            True           511
Lisbon      False         1420
            True           690
Lyon        False         3283
            True          1580
Madrid      False         7009
            True          2222
Malmö       False         1601
            True

In [110]:
occurence_customer

customer
Amsterdam     5413
Athens        3143
Barcelona     4386
Berlin        9819
Bordeaux      1621
Bremen        1612
Bucharest     3352
Budapest      2720
Cologne       3904
Copenhagen    4877
Hanover       2250
Helsinki      1588
Lisbon        2110
Lyon          4863
Madrid        9231
Malmö         2163
Marseille     3842
Milan         9670
Munich        6047
Naples        2680
Paris         6606
Porto         1112
Prague        1620
Rome          3860
Stockholm     4926
Turin         2143
Valencia      1638
Vienna        7080
dtype: int64

In [ ]:
data = pd.read_csv("datas/orders.csv", sep=";")


In [101]:
data

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units,late_order
0,366c7a3d298f,Rotterdam,v_002,DTP,Venlo,Marseille,1692723,583,True
1,45f906331e10,Rotterdam,v_004,CRF,Rome,Marseille,1644308,459,False
2,ac84a6e4af0f,Athens,v_002,CRF,Venlo,Paris,1684170,464,True
3,f5e98cb29790,Rotterdam,v_004,CRF,Lille,Milan,1620510,678,False
4,a9e7c9bee35b,Barcelona,v_002,CRF,Venlo,Berlin,1699372,353,False
...,...,...,...,...,...,...,...,...,...
114271,3f4b15fb770e,Rotterdam,v_002,CRF,Dusseldorf,Bordeaux,1681376,645,False
114272,d2e6978a38fd,Barcelona,v_004,DTD,Dusseldorf,Berlin,1676942,502,False
114273,b88babd5c7bd,Rotterdam,v_002,DTP,Dusseldorf,Rome,1692737,464,False
114274,b0b5c761613f,Barcelona,v_003,DTD,Dusseldorf,Munich,1699974,388,False
